# Quantitative Risk Management

Click <a href="https://colab.research.google.com/github/Lolillosky/QuantRiskManagement/blob/main/NOTEBOOKS/9_Vol_VaR_CVaR.ipynb">
    <img src="https://upload.wikimedia.org/wikipedia/commons/d/d0/Google_Colaboratory_SVG_Logo.svg" width="30" alt="Google Colab">
</a> to open this notebook in Google Colab.


In this notebook you will calculate sensitivities, VaR and CVaR of a portfolio under different assumptions.

The portfolio will be comprised of the following:

* +1 share and a long put option with strike $125$ on A.
* +1 share and a short call option with strike $150$ on B.
* A short call option with strike $135$ on C.
 
Every option has a maturity equal to 6 months. Value date is 07-12-2022 (last date in the data frame). Assume risk free rate is $1\%$ and dividend yields $0\%$. Give an estimate of the sensitivities, volatility, value at risk and conditional value at risk of your portfolio for a ten day horizon. VaR and CVaR should have a $97.5\%$ confidence level. 

Volatilities are in percentage (should be divided by $100$). Use the Black Scholes implementation provided in the repository.


## Import Main Libraries 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch

## Import Libraries from Github Repository 

In [ ]:
import sys
sys.path.append('../CODE')  # Adjust the path as necessary


from IPython.display import clear_output

!rm -r {'QuantRiskManagement'}

!git clone https://github.com/Lolillosky/QuantRiskManagement.git

import sys
sys.modules.pop
sys.path.insert(0,'QuantRiskManagement/CODE')

clear_output()


In [ ]:
import Option_formulas
import pytorch_option_formulas
import portfolio_npv_calculations

## Read Historica Data

In [ ]:
#risk_factors = pd.read_csv('../DATA/Histdata_equity.csv',index_col=0,parse_dates=True)
risk_factors = pd.read_csv('/content/QuantRiskManagement/DATA/Histdata_equity.csv',index_col=0,parse_dates=True)

risk_factors['Vol A'] /= 100
risk_factors['Vol B'] /= 100
risk_factors['Vol C'] /= 100

risk_factors.head()

## Base Scenario Delta Calculations

In [ ]:
notionals = np.array([1.0, 1.0, 1.0, -1.0, -1.0])
notionals_torch = torch.tensor(notionals, requires_grad=True)

base_scenario = risk_factors.values[-1].reshape(1,-1)
base_scenario_torch = torch.tensor(base_scenario, requires_grad=True)

portfolio_torch = [lambda t, risk_factors: risk_factors[:,0],
            lambda t, risk_factors: pytorch_option_formulas.BlackScholes(risk_factors[:,0], torch.tensor(125.0),
                torch.tensor(0.5)-t, torch.tensor(0.01), torch.tensor(0.0), risk_factors[:,1], False),
            lambda t, risk_factors: risk_factors[:,2],
            lambda t, risk_factors: pytorch_option_formulas.BlackScholes(risk_factors[:,2], torch.tensor(150.0),
                torch.tensor(0.5)-t, torch.tensor(0.01), torch.tensor(0.0), risk_factors[:,3], True),
            lambda t, risk_factors: pytorch_option_formulas.BlackScholes(risk_factors[:,4], torch.tensor(135.0),
                torch.tensor(0.5)-t, torch.tensor(0.01), torch.tensor(0.0), risk_factors[:,5], True)]

portfolio = [lambda t,risk_factors: risk_factors[:,0],
            lambda t,risk_factors: Option_formulas.BlackScholes(risk_factors[:,0], 125,0.5-t,0.01, 0.0, risk_factors[:,1], False),
            lambda t,risk_factors: risk_factors[:,2],
            lambda t,risk_factors: Option_formulas.BlackScholes(risk_factors[:,2], 150,0.5-t,0.01, 0.0, risk_factors[:,3], True),
            lambda t,risk_factors: Option_formulas.BlackScholes(risk_factors[:,4], 135,0.5-t,0.01, 0.0, risk_factors[:,5], True)]



calculator_torch = portfolio_npv_calculations.Portfolio_Delta_NPV_Calculator(notionals_torch,portfolio_torch, option = 'torch')
calculator = portfolio_npv_calculations.Portfolio_Delta_NPV_Calculator(notionals,portfolio, option = 'numpy')

NPV = calculator_torch.value(torch.tensor(0.0),base_scenario_torch)[0]

print(NPV)

NPV.backward()

delta_vega = base_scenario_torch.grad

print(delta_vega)

## Full Valuation Scenarios

We will use 10-days overlapping historical scenarios so that for risk factor $\theta_j$ and scenario i:

$$\theta_j^i=\theta_j^0\frac{\theta_j^{t_i}}{\theta_j^{t_{i-10}}}$$

In [ ]:
horizon = 10


risk_factors_shocked = base_scenario* risk_factors.values[horizon:]/risk_factors.values[:-horizon]
        

portfolio_pl, constituents_pl = calculator.compute_scenarios_pl(base_scenario,0,risk_factors_shocked)


plt.hist(portfolio_pl,bins=100);

## Volatility

Let's do a quick check: for the whole set of scenarios, compute the volatility using numpy np.std function:

In [ ]:
# Your code here

Compare the calculation with the covariance matrix formula:

In [ ]:
# Your code here

Now compute the marginal contribution to volatility with the formula

In [ ]:
# Your code here

Now compute the marginal contribution to volatility with Pytorch

In [ ]:
# Your code here

With a rolling window of 250 scenarios over the 10 day historical shocks, compare the volatility obtained with full valuation with that obtained with sensitivities and with that obtained by the delta normal approach.  

In [ ]:
# Your code here

## Value at Risk

For the whole set of scenarios, compute VaR using numpy:

In [ ]:
# Your code here

Now compute the marginal contribution to VaR with Pytorch

In [ ]:
# Your code here

With a rolling window of 250 scenarios over the 10 day historical shocks, compare the VaR obtained with full valuation with that obtained with sensitivities and with that obtained by the delta normal approach.  

In [ ]:
# Your code here

## Conditional Value at Risk

For the whole set of scenarios, compute CVaR using numpy:

In [ ]:
# Your code here

Now compute the marginal contribution to CVaR with Pytorch

In [ ]:
# Your code here

With a rolling window of 250 scenarios over the 10 day historical shocks, compare the CVaR obtained with full valuation with that obtained with sensitivities and with that obtained by the delta normal approach.  

In [ ]:
# Your code here